In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [3]:
def feature_eng(train):
    # train = df_in.copy()
    #Replacing NA values in Bed Grade Column for both Train and Test datssets
    train['Bed Grade'].fillna(train['Bed Grade'].mode()[0], inplace = True)

    #Replacing NA values in  Column for both Train and Test datssets
    train['City_Code_Patient'].fillna(train['City_Code_Patient'].mode()[0], inplace = True)

    # Label Encoding Stay column in train dataset
    le = LabelEncoder()
    # train["Stay"] = le.fit_transform(train["Stay"].astype('str'))

    #Label Encoding all the columns in Train and test datasets
    for i in ['Hospital_type_code', 'Hospital_region_code', 'Department',
            'Ward_Type', 'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness', 'Age']:
        le = LabelEncoder()
        train[i] = le.fit_transform(train[i].astype(str))

    return train

In [10]:
df_los = pd.read_csv("los.csv")
df_los.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50


In [11]:
feature_eng(df_los).drop(['case_id', 'patientid', 'Hospital_region_code', 'Ward_Facility_Code'], axis =1).head()

,Hospital_code,Hospital_type_code,City_Code_Hospital,Available Extra Rooms in Hospital,Department,Ward_Type,Bed Grade,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,8,2,3,3,3,2,2.0,7.0,0,0,2,5,4911.0,0-10
1,2,2,5,2,3,3,2.0,7.0,1,0,2,5,5954.0,41-50
2,10,4,1,2,1,3,2.0,7.0,1,0,2,5,4745.0,31-40
3,26,1,2,2,3,2,2.0,7.0,1,0,2,5,7272.0,41-50
4,26,1,2,2,3,3,2.0,7.0,1,0,2,5,5558.0,41-50


In [ ]:
def get_countid_enocde(train, cols, name):
    temp = train.groupby(cols)['case_id'].count().reset_index().rename(columns = {'case_id': name})
    train = pd.merge(train, temp, how='left', on= cols)
    train[name] = train[name].astype('float')
    train[name].fillna(np.median(temp[name]), inplace = True)
    return train

def test_train_split(df):  
    #Spearating Train and Test Datasets
    train = df[df['Stay']!=-1]
    test = df[df['Stay']==-1]

    train = get_countid_enocde(train, ['patientid'], name = 'count_id_patient')
    train = get_countid_enocde(train, 
                                    ['patientid', 'Hospital_region_code'], name = 'count_id_patient_hospitalCode')
    train = get_countid_enocde(train, 
                                    ['patientid', 'Ward_Facility_Code'], name = 'count_id_patient_wardfacilityCode')

    # Droping duplicate columns
    test1 = test.drop(['Stay', 'patientid', 'Hospital_region_code', 'Ward_Facility_Code'], axis =1)
    train1 = train.drop(['case_id', 'patientid', 'Hospital_region_code', 'Ward_Facility_Code'], axis =1)

    # Splitting train data for Naive Bayes and XGBoost
    X1 = train1.drop('Stay', axis =1)
    y1 = train1['Stay']
    X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size =0.20, random_state =100)

    return X_train, X_test, y_train, y_test